<a href="https://colab.research.google.com/github/Tenrog/CSC3916_HW0/blob/Patrick's-Branch/NREL_Spring_2022_Penultimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implementing Random Sketching in GMRes**

#Work Breakdown

* Em Gibbs: Main writer, coder. Organized and composed the notebook. Researched sparse matrix sketching. Tested Spring 2021 sketching functions. Assisted with RGMRes coding.
* Haonan He: Manager, coder. Researched random skething, RGS Arnoldi, and randomized GMRes. Wrote RGMRes code implmentation.
* Patrick Gornet: Main coder, communicator. Led communication with sponsor. Researched classical GMRes. Wrote classical GMRes implementations. Assisted with RGMRes coding.

#Abstract

When solving linear systems of equations in the form $Ax = b$ where $A$ and $b$ are given, finding the exact value of $x$ for a large $A$ can be extremely inefficient. Iterative methods like GMRes can speed up this computation time by finding $x$ values that satisfy the equation within a specified margin of error, but are still slow. By introducing random sketching into GMRes, in which $n$ dimensions are randomly embedded into $k$ dimensions such that $k \ll n$, this project hopes to greatly reduce the computation time of $x$ without loss of accuracy.

#Introduction

* The motivation of this project is to accelerate the computation time of GMRes using the process of random sketching on some of the matrices (and vectors) used in the algorithm. By using smaller matrices sketched from the originals, computation time of each iteration of the GMRes algorithm is dramatically reduced, and similar levels of accuracy should be achievable.
* The National Renewable Energy Laboratory (NREL), the sponsor of this project, often works with $A$ matrices with dimensions in the billions. Reducing one of those dimensions by a significant factor will more significantly reduce this computation time.
* Specifically, this project aims to implement the Randomized Gram-Schmidt algorithm (RGS) described in "Randomized Gram-Schmidt process with application to GMRES" by Grigori and Balabanov, where the Modified Gram-Schmidt algorithm (MGS) would otherwise be used in GMRes.
* This project therefore will consider pre-existing implementations and pseudocode of GMRes, modifying them or constructing new implementations suited to integrating the random sketching methods defined in the Grigori & Balabanov paper.
* When an implementation of GMRes with random sketching is fully developed, this project will test this implemenation against other GMRes implementations for computational time and accuracy.

#Methods

##Imports

In [ ]:
import numpy as np
import math
import time
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import gmres
from scipy.sparse.linalg import lgmres
from scipy.sparse import random as rand_sparse

## Classical GMRes Implementations

###SciPy GMRes

Given the use of Python in developing an implementation of randomized GMRes (RGMRes), we considered the SciPy package, whose linear algebra package includes an implementation of GMRes.

The following code tests the runtime of SciPy's GMRes for matrices of size $100 \times 100$, $750 \times 750$, and $1250 \times 1250$, as well as the number of iterations, and error of their results. These will form a baseline for comparison developing further implementations.

Note, for the purpose of counting iterations, the following gmres_counter() function was taken from [Stack Overflow](https://stackoverflow.com/questions/33512081/getting-the-number-of-iterations-of-scipys-gmres-iterative-method) and will be used later as well.

In [ ]:
class gmres_counter(object):
    def __init__(self, disp=True):
        self._disp = disp
        self.niter = 0
    def __call__(self, rk=None):
        self.niter += 1
        return self.niter

In [ ]:
#array of sizes
size = [100, 750, 1250]

#loop through sizes
for curr_size in size:
  #variable declaration
  A = np.random.rand(curr_size,curr_size)
  xTrue = np.random.rand(curr_size)
  xNorm = np.sqrt(np.average((xTrue)**2))
  b = np.dot(A,xTrue)
  counter = gmres_counter()
  x_0 = np.zeros(curr_size)

  #timing GMRes and calculating error
  t1 = time.perf_counter()
  x = gmres(A,b, x_0, restart=1000, callback = counter)[0]
  t2 = time.perf_counter()
  error = np.sqrt(np.average((xTrue - x)**2))/xNorm

  print("For", curr_size, "x", curr_size, ":")
  print("Error:", error)
  print("Iterations:", counter()-1)
  print("Time: ", t2 - t1)

For 100 x 100 :
Error: 6.6888008892898445e-15
Iterations: 100
Time:  0.008965910000370059
For 750 x 750 :
Error: 0.04634160882068907
Iterations: 749
Time:  0.38924825899994175
For 1250 x 1250 :
Error: 0.48458613291381025
Iterations: 12500
Time:  15.060490763999951


The following code does the same as the above code, but with sparse matrices of density 0.1.

In [ ]:
#array of sizes
size = [100, 750, 1250]

#loop through sizes
for curr_size in size:
  #variable declaration
  A = rand_sparse(curr_size, curr_size, density=0.1).toarray()
  xTrue = np.random.rand(curr_size)
  xNorm = np.sqrt(np.average((xTrue)**2))
  b = np.dot(A,xTrue)
  counter = gmres_counter()
  x_0 = np.zeros(curr_size)

  #timing GMRes and calculating error
  t1 = time.perf_counter()
  x = gmres(A,b, x_0, restart=1000, callback = counter)[0]
  t2 = time.perf_counter()
  error = np.sqrt(np.average((xTrue - x)**2))/xNorm

  print("For", curr_size, "x", curr_size, ":")
  print("Error:", error)
  print("Iterations:", counter()-1)
  print("Time: ", t2 - t1)

For 100 x 100 :
Error: 5.797220796177716e-14
Iterations: 100
Time:  0.0041206249998140265
For 750 x 750 :
Error: 4.297211446240082e-14
Iterations: 750
Time:  0.39393633400004546
For 1250 x 1250 :
Error: 0.5225190666080766
Iterations: 12500
Time:  14.81432392700026


Given these relatively fast implementations implementations of GMRes, we would like to build off of them with sketching, but their source code is more complicated than necessary for the purposes of this project.

###Wikipedia MatLab to Python GMRes

Another implementation of GMRes considered was that provided by the Wikipedia page for GMRES. This page provides the code for a [MatLab implementation](https://en.wikipedia.org/wiki/Generalized_minimal_residual_method#Regular_GMRES_(MATLAB_/_GNU_Octave), and given MatLab's similarity to Python, we considered translating this code to Python, with the following results.

In [ ]:
# This function assumes that inputs are numpy arrays
def gmresWiki( A, b, x, max_iterations, threshold):
  n = max(np.shape(A))
  m = max_iterations

  # use x as the initial vector
  r = np.subtract(b, A.dot(x))

  b_norm = np.linalg.norm(b)
  error = np.divide(np.linalg.norm(r), b_norm)

  # initialize the 1D vectors
  sn = np.zeros((m, 1))
  cs = np.zeros((m, 1))
  #e1 = np.zeros((n, 1))
  e1 = np.zeros((m+1, 1))
  e1[0] = 1
  e = np.array([error])
  r_norm = np.linalg.norm(r)

  Q = np.reshape(np.array(np.divide(r, r_norm)),(-1,1))

  beta = np.multiply(r_norm, e1)     #Note: this is not the beta scalar in section "The method" above but the beta scalar multiplied by e1
  
  for k in range(m):
    print(k)
    # run arnoldi

    htemp, qtemp= arnoldiWiki(A, Q, k)
    H = np.hstack(H, np.transpose(htemp))
    Q = np.vstack(Q, qtemp)
    
    # eliminate the last element in H ith row and update the rotation matrix
    htemp, cs[k,0], sn[k,0], = apply_givens_rotation(H[:,k], cs, sn, k)
    H = np.hstack(H, np.transpose(htemp))
    # update the residual vector
    beta[k + 1, 0] = -sn[k,0] * beta[k, 0]
    beta[k, 0]     = cs[k,0] * beta[k, 0]
    error       = abs(np.divide(beta[k + 1, 0], b_norm))

    # save the error
    e = np.vstack(e, error)

    if (error <= threshold):
      break
    
  
  # if threshold is not reached, k = m at this point (and not m+1) 
  
  # calculate the result
  y = np.divide(H,beta)
  x = x + Q[:,0:k] * y
  return x, e


#Arnoldi Function

def arnoldiWiki(A, Q, k):
    q = np.multiply(A,Q[:,k])   # Krylov Vector
    h = np.zeros((1,k))
    for i in range(k):    # Modified Gram-Schmidt, keeping the Hessenberg matrix
        h[0,i] = np.multiply(np.transpose(q), Q[:, i])
        q = np.subtract(q, np.multiply(h[0:i], Q[:, i]))
    print(np.linalg.norm(q, axis = 0)[1,1])
    h = np.append(h, [[np.linalg.norm(q)]])
    q = np.divide(q, h[0, k + 1])
    return h, q


#Applying Givens Rotation to H col

def apply_givens_rotation(h, cs, sn, k):
  # apply for ith column
    for i  in range(k-1):
        temp   = np.add(np.multiply(cs[i, 0], h[i, 0]), np.multiply(sn[i, 0], h[i + 1, 0]))
        h[i + 1, 0] = np.add(np.multiply(-sn[i, 0], h[i, 0]), np.multiply(cs[i, 0], h[i + 1, 0]))
        h[i, 0]  = temp
  
    # update the next sin cos values for rotation
    cs_k, sn_k = givens_rotation(h[k, 0], h[k + 1, 0])

    # eliminate H(i + 1, i)
    h[k, 0] = np.add(np.multiply(cs_k, h[k, 0]), np.multiply(sn_k, h[k + 1, 0]))
    h[k + 1, 0] = 0.0
    return h, cs_k, sn_k

#Calculate the Givens rotation matrix
def givens_rotation(v1, v2):

    t = np.sqrt(np.add(np.power(v1, 2), np.power(v2, 2)))
    cs = np.divide(v1, t) 
    sn = np.divide(v2, t)
    return cs, sn

While the above code does not run in its current state, the exercise of translating it was valuable in building understanding of GMRes. Unfortunately, the use of the Givens rotation made it incompatible with RGMRes as defined by the Balabanov & Grigori paper, so it is not suitable for building upon.

###Sauer Numerical Analysis GMRes

Finally, we considered the GMRes pseudocode as defined by the Numerical Analysis 3rd Edition textbook by Timothy Sauer. The pseudocode is as follows:

$x_{0} =$ initial guess

$r = b - Ax_{0}$

$q_{1} = r / ||r||_{2}$

**for** $k = 1, 2, ..., m$

>$y = Aq_{k}$

>**for** $j = 1, 2, ..., k$

>>$h_{jk} = q_{j}^{T}y$

>>$y = y - h_{jk}q_{j}$

>**end**

>$h_{k+1,k} = ||y||_{2}$ (If $h_{k+1,k} = 0$, skip next line and terminate at bottom.)

> $q_{k+1} = y/h_{k+1,k}$

> Minimize $||Hc_{k} - [||r||_{2} \: 0 \:0 \: ... 0]^{T}||_{2}$ for $c_{k}$

>$x_{k} = Q_{k}c_{k} + x_{0}$

**end**

Given the above pseudocode, the following implementation was developed.

In [ ]:
def leastSqr (H, b_temp):
    c = np.linalg.lstsq(H,b_temp, rcond=None)[0]
    return c

def SauerGMRes (A, b, x0, max_iterations = 1000, threshold = 0.000001):
    max_iterations = min(max_iterations, max(np.shape(A)))
    # Ax = b
    # A in input matrix
    # b is vector
    # x0 is initial guess vector
    # r = b - Ax_0
    r = np.subtract(b, np.asarray(np.dot(A, x0)).reshape(-1))
    x = []
    Q = [0] * (max_iterations)
    #calculate forward and backward error
    #ensure backward error is going down
    x.append(r)

    # q1 = r/||r||_2
    Q[0] = np.divide(r, np.linalg.norm(r))

    H = np.zeros((max_iterations + 1, max_iterations)) #have H expand and initilize b here
    steps = 0
    #Iterate
    for k in range(max_iterations):
        # y = Aq_k
        y = np.asarray(np.dot(A, Q[k])).reshape(-1)
        
        for j in range(k):
            # h_(jk) = q_j^T*y
            H[j, k] = np.dot(Q[j], y)
            # y = y - h_(jk)q_j
            y = np.subtract(y, np.multiply(H[j, k], Q[j]))
        
        # h_(k+1,k) = ||y||_2
        H[k + 1, k] = np.linalg.norm(y)
        # Checking if h_(k+1,k) is 0
        if (H[k + 1, k] != 0 and k != max_iterations - 1):
          # q_(k+1) = y/h_(k+1,k)
          Q[k + 1] = np.divide(y, H[k + 1, k])
        else:
          steps = k
          break
        
        # Creating vector [||r||_2, 0, 0, ..., 0]^T
        b_temp = np.zeros(max_iterations + 1) #possiable problem, should be k
        b_temp[0] = np.linalg.norm(r)
        # Minimize ||Hc_k - [||r||_2, 0, 0, ..., 0]^T||_2 for c_k
        c = leastSqr(H, b_temp)
        # x_k = Q_kc_k+x_0
        x.append(np.add(np.dot(np.asarray(Q).transpose(), c), x0))
    
    
    return x[-1], steps

The following code tests this implementation of GMRes against SciPy.

Note: At time of last execution, it took over 150 seconds for the Sauer GMRes implementation to solve the problem with an A matrix of size $750 \times 750$, and it did not solve the $1250 \times 1250$ problem.

In [ ]:
#array of sizes
#size = [100, 750, 1250]
size = [100]

#loop through sizes
for curr_size in size:
  #variable declaration
  A = np.random.rand(curr_size, curr_size)
  xTrue = np.random.rand(curr_size)
  xNorm = np.sqrt(np.average((xTrue)**2))
  b = np.dot(A,xTrue)
  counter = gmres_counter()
  x_0 = np.zeros(curr_size)

  #timing SciPy GMRes and calculating error
  t1 = time.perf_counter()
  x = gmres(A,b, x_0, restart=1000, callback = counter)[0]
  t2 = time.perf_counter()
  error = np.sqrt(np.average((xTrue - x)**2))/xNorm

  print("For", curr_size, "x", curr_size, ":")
  print("SciPy GMRes")
  print("Error:", error)
  print("Iterations:", counter()-1)
  print("Time: ", t2 - t1)

  #timing Sauer GMRes and calculating error
  t1 = time.perf_counter()
  x, k = SauerGMRes(A, b, x_0)
  t2 = time.perf_counter()
  error = np.sqrt(np.average((xTrue - x)**2))/xNorm

  print("Sauer GMRes")
  print("Error:", error)
  print("Iterations:", k)
  print("Time: ", t2 - t1)

  print()

For 100 x 100 :
SciPy GMRes
Error: 1.9660669632128094e-14
Iterations: 100
Time:  0.004272920999937924


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Sauer GMRes
Error: 0.4003400942464691
Iterations: 99
Time:  0.26527165500010597



Given the Sauer GMRes implementation takes much longer than SciPy's, we can aniticipate that introducing sketching will reduce runtime significantly from the classical implementation.

##Random Sketching

###Important Functions

The following functions are taken from the Spring 2021 NREL Math Clinic project, and are important for testing later in this section and elsewhere in the notebook.

In [ ]:
#Defining important functions

#Defining matrix multiplication function
def mat_mul(x,y):
  return np.matmul(x,y) #AF: coders asked to write a faster code based on Θ structure

#Building Θ Matrix
def Theta_Matrix(k,n): #AF: brute force implementation
  Theta = (np.random.randint(0,2,size=(k,n))*2-1)*math.sqrt(k)**-1
  return Theta

#Finds smallest possible k from inequality 3a using values set above.
def mink(ϵ, d, δ):                           # δ, ϵ are local here
  min_k = 7.87*ϵ**-2*(6.9*d + math.log(1/δ)) #AF: Grigori & Balabanov (3a)
  return min_k
  
def xIterCompare(xTrue, xOld, xNew):
  xTrueNorm = np.sqrt(np.average((xTrue)**2))
  oldError = np.sqrt(np.average((xTrue - xOld)**2))/xTrueNorm
  newError = np.sqrt(np.average((xTrue - xNew)**2))/xTrueNorm
  return oldError, newError

def xAccuracy(xTrue, x):
  xTrueNorm = np.sqrt(np.average((xTrue)**2))
  error = np.sqrt(np.average((xTrue - x)**2))/xTrueNorm
  return error

###Balabanov & Grigori Inequality 2.1

The following inequality is defined as Definition 2.1 in the Balabanov and Grigori paper (i.e., the definition of an ϵ-subspace embedding for V):
$$∀x, y∈V; |<x;y> - <Θx;Θy>| ≤ ε||x||⋅||y||$$ 

For a sketching matrix Θ, δ is equal to the failure rate of inequality 2.1.

The following code blocks test various random vectors x and y from V to determine a failure rate δ for inequality 2.1. The function inequality2 is taken from the Spring 2021 NREL Math Clinic project.

By choosing and varying k and d in the generation of the Θ matrix, we hope to discover ε and δ values using the following inequality 2.2a:

$$k≥7.87ε^{-2}(6.9d + log(1/δ))$$

In [ ]:
def inequality2(V, d, Θ, ϵ):
  #Getting x and y arrays in V subspace
  a = np.random.normal(0,1,d) 
  x_from_V =  mat_mul(V, a)
  b = np.random.normal(0,1,d) 
  y_from_V = mat_mul(V, b)
  #Matrix vector products of Θ and x,y
  theta_x = mat_mul(Θ, x_from_V)
  theta_y = mat_mul(Θ, y_from_V)
  #Calculate right and left hand side of the equation
  LHS = abs(np.inner(x_from_V,y_from_V) - np.inner(theta_x, theta_y))
  RHS = ϵ*np.linalg.norm(x_from_V)*np.linalg.norm(y_from_V)
  #Comparing RHS to LHS   
  if LHS <= RHS:  #AF: I asked to return and inspect (RHS – LHS) to see how close the success is
    return True
  else:
    return False

#Test function for inequality 2
def test_inequality2(num_iters, V, d, Θ, ϵ):
  passes = 0
  failures = 0
  for i in range(num_iters):
    if inequality2(V, d, Θ, ϵ):
      passes += 1
    else:
      failures += 1

  return failures / (failures + passes)

In [ ]:
#assign variables
n = 5000
ϵ = 1/2
d = [5, 50, 500]
k = [500, 1000, 2000, 4000]

for d_iter in d:
  for k_iter in k:
    #generate theta
    Θ = Theta_Matrix(k_iter, n)
    V = np.random.normal(0, 1, (n, d_iter))

    #iterate test inequality 2.1 100 times
    iter = 100
    failrate = test_inequality2(iter, V, d_iter, Θ, ϵ)

    print("For k = ", k_iter, ", d = ", d_iter, ":")
    print("Failure rate = ", failrate)

For k =  500 , d =  5 :
Failure rate =  0.0
For k =  1000 , d =  5 :
Failure rate =  0.0
For k =  2000 , d =  5 :
Failure rate =  0.0
For k =  4000 , d =  5 :
Failure rate =  0.0
For k =  500 , d =  50 :
Failure rate =  0.0
For k =  1000 , d =  50 :
Failure rate =  0.0
For k =  2000 , d =  50 :
Failure rate =  0.0
For k =  4000 , d =  50 :
Failure rate =  0.0
For k =  500 , d =  500 :
Failure rate =  0.0
For k =  1000 , d =  500 :
Failure rate =  0.0
For k =  2000 , d =  500 :
Failure rate =  0.0
For k =  4000 , d =  500 :
Failure rate =  0.0


The Spring 2021 NREL Math Clinic team also tested several possible values for these variables in the inequality 2.1, but could never get the inequality to fail as long as other necessary inequalities for the variables still held. This seemed strange, as we then have no basis for the selection of δ. However, we can see that these variable tests also produced no failures of the inequality, so we will continue to use the Spring 2021 chosen variable values (δ of 0.9 and ε of 1/2).

###Testing Spring 2021 Randomized Gram-Schmidt

The following functions are taken from the Spring 2021 NREL Math Clinic project. This code block has an implementation of Gram-Schmidt taken from Professor Liu, and their implementation of Randomized Gram-Schmidt (RGS).

In [ ]:
'(Liu)'
'Used to compare RGS with classic GS'
def reduced_QR(A):
  """
  Perform reduced QR factorization with Gram Schmidt orthogonalization
  for the columns of Areturn Q: a matrix of size m x n with orthonormal columns
  R: an upper triagular matrix of size n x n
  """
  n, m = A.shape                          #AF: m, n swapped cf. rest of code
  Q = np.zeros((n,m))
  R = np.zeros((m,m))
  for i in range(m):
    y = A[:, i]
    for j in range(i):
      R[j,i] = np.dot(Q[:,j], A[:,i])
      y = y - R[j,i]*Q[:,j]
    R[i,i] = np.linalg.norm(y)
    Q[:,i] = y/R[i,i]

    #print("Reduced QR")
    #print("R[i,i]", R[i,i])
  return Q, R

def rand_QR(A, k):
  #Setting Values                           #AF: B&G Algo. 2 requires m ≤ k << n
  m = A.shape[1]
  n = A.shape[0]
  Θ = Theta_Matrix(k,n)
  #Setting zero matrices to build
  Q = np.zeros((n,m))
  R = np.zeros((m,m))                       #AF: could use sparse structure
  S = np.zeros((k,m))                       #AF: S need not be formed
  p = np.dot(Θ, A)                          #AF: p need not be formed
  for i in range(m):                        #AF: So i here is i–1 in RGS_balabanov_grigori_2020.pdf
    #steps 2, 3:
    if i == 0:
      #step 3
      q_prime_i = A[:,i]
    else:
      new_holder = S[:,i-1]                 #AF: B&G Algo. 2 says S[:,:i] not S[:,i-1]
      new_holder = np.reshape(new_holder, (k,1))
      holder = np.linalg.pinv(new_holder)   #AF: the pinv of a tall S is always (S⸆S)⁻¹S⸆
      r_i = np.dot(holder, p)               #AF: np.linalg.solve(S⸆S,S⸆p) would be faster
      R[i-1,:] = r_i                        #AF: not indexed like B&G Algo. 2 step 2
      q_prime_i = A[:,i] - np.dot(Q, R[:,i])#AF: not indexed like B&G Algo. 2 step 3
    #step 4
    s_prime_i = np.dot(Θ, q_prime_i)
    #step 5
    r = np.linalg.norm(s_prime_i)
    #step 6
    s_i = s_prime_i/r
    #step 7
    q_i = q_prime_i/r
    #Building Q, R, S
    Q[:,i] = q_i
    S[:,i] = s_i
    if i==(m-1):                            #AF: this whole block is some ghastly kluge against above errors
      new_holder = S[:,i]
      new_holder = np.reshape(new_holder, (k,1))
      holder = np.linalg.pinv(new_holder)
      r_i = np.dot(holder, p)
      R[i,:] = r_i
      for k in range(m):
        R[k+1:,k] = 0
  return Q, R

Additionally, the following function RGS_testing was also taken from the Spring 2021 project, but modified slightly.

In [ ]:
#Numerical Stability Test
#iter is number of iterations
def RGS_testing(iter, A, k):
  results = []
  times = []
  for i in range(iter):
    #Performing the QR decompositions and tracking the time
    tic = time.perf_counter()
    GS_Q, GS_R = reduced_QR(A)
    toc = time.perf_counter()
    GS_time = toc - tic
    tic = time.perf_counter()
    RGS_Q, RGS_R = rand_QR(A, k)
    toc = time.perf_counter()
    RGS_time = toc - tic
    time_diff = GS_time - RGS_time
    #Finding and comparing the accuracy of the two methods
    GS_results = A - mat_mul(GS_Q, GS_R)
    GS_Ortho = np.identity(np.shape(A)[1]) - GS_Q.T @ GS_Q
    RGS_results = A - mat_mul(RGS_Q, RGS_R)
    RGS_Ortho = np.identity(np.shape(A)[1]) - RGS_Q.T @ RGS_Q
    GS_norm = np.linalg.norm(GS_results)/np.linalg.norm(A)
    RGS_norm = np.linalg.norm(RGS_results)/np.linalg.norm(A)
    GS_Onorm = np.linalg.norm(GS_Ortho)/(np.sqrt(np.shape(A)[1]))
    RGS_Onorm = np.linalg.norm(RGS_Ortho)/(np.sqrt(np.shape(A)[1]))
    accuracy = (GS_norm, RGS_norm, GS_Onorm, RGS_Onorm)
    #Setting results
    results.append(accuracy)
    times.append(time_diff)
  return results, times

The following code was slightly modified from the Spring 2021 project and comapres runtime and accuracy of GS and their implementation of RGS.

In [ ]:
#variable declarations
n = 1000
m = 500
k = 750

A = np.random.rand(n, m)
iter = 3

#run tests
results, times = RGS_testing(iter, A, k)

#print results
for i in range(iter):
  print("On iteration", i)
  print("The accuracy difference was: " + str(results[i]))
  print("The time difference was: " + str(times[i]))
  if times[i] < 0:
    print("Gram Schmidt was quicker than RGS here.")
  elif times[i] == 0:
    print("RGS and GS took the same time here.")
  else:
    print("RGS was quicker than GS here.")
  print("\n")

On iteration 0
The accuracy difference was: (6.638410798977528e-16, 3.5330340197700863e-15, 1.7947338713753687e-14, 4.7265790389175155)
The time difference was: 3.734614910000005
RGS was quicker than GS here.


On iteration 1
The accuracy difference was: (6.638410798977528e-16, 3.126121033919637e-15, 1.7947338713753687e-14, 4.742457005947366)
The time difference was: 3.2764480959999958
RGS was quicker than GS here.


On iteration 2
The accuracy difference was: (6.638410798977528e-16, 3.092920635854113e-15, 1.7947338713753687e-14, 4.757408615360058)
The time difference was: 3.3479587459999607
RGS was quicker than GS here.




Although the RGS function runs much faster than the MGS function, the accuracy is not as good. Without spending more time working with the code, it is hard to say what is causing this inflated error.

While the Spring 2021 NREL Math Clinic team managed to develop a working Randomized Gram-Schmidt function, there are issues with the accuracy that make it worrisome to implement. Additionally, the function is not constructed in a way that is conducive to implementing it into a full RGMRes. Rather than trying to debug and modify the code to work for us, we opted to develop our own implementation of the full RGMRes as described in the Grigori & Balabanov paper.

###Testing Algo_2

The following function my_sketch, allows us to define whether to run our algorithm 2 and RGMRes implementations with sketching or not.

In [ ]:

sketch = False
if (sketch):
  def my_sketch(sketch_mat, mat):
    return sketch_mat @ mat
else:
    def my_sketch(sketch_mat, mat):
      return mat


We developed our own implementation of randomized GMRes. This function is defined as algo_2. The following code defines algo_2, as well as a modified RGS_testing function for comparison.

In [ ]:
#Balabanov & Grigori Algorithm 2 (RGS)
#Parameters: nxm array W, kxn array sketch_mat, boolean sketch
#Returns: nxm factor Q, mxm upper triangular factor R
def algo_2(W, sketch_mat):
    #Getting sizes of matrices
    n, m = W.shape
    k = len(sketch_mat)

    #Initializing zero matrices
    S = np.zeros((n,m))
    S = my_sketch(sketch_mat, S)
    Q = np.zeros((n,m))
    R = np.zeros((m,m))

    #RGS Algorithm
    for i in range(m):
        #2.1: Sketch w_i: p_i = Θw_i
        p = my_sketch(sketch_mat, W[:,i])

        q_p = W[:,i]
        #2.2: Solve k x (i-1) least-squares problem:
        #   R_1:i-1,i = arg min_y ||(S_i-1)y - p_i||
        #   lstsq method
        #R[:i-1,i] = np.linalg.lstsq(S[:,:i-1], p)[0]
        #   pinv method
        #S_pinv = np.linalg.pinv(S[:,:i-1])
        #R[:i-1,i] = np.dot(S_pinv,p)

        """
        print()
        print("At RGS iteration ", i)
        print(i-1)
        print("R[:i-1,i] after lstsq")
        print(R[:i-1,i])
        print("p")
        print(p)
        print("S[:,:i-1]")
        print(S[:,:i-1])
        print()
        """

        #   fournier method
        S_sub = S[:,:i]
        StS = np.dot(S_sub.transpose(),S_sub)
        Stp = np.dot(S_sub.transpose(), p)
        R[:i,i] = np.linalg.solve(StS, Stp)
        #   liu method
        #for j in range(i):
        #    R[j,i] = np.dot(Q[:,j], W[:,i])
        #    q_p = q_p - R[j,i]*Q[:,j]
        #2.3: Compute projection of w_i: q_i_p = w_i - (Q_i-1)R_1:i-1,i
        q_p = q_p - np.dot(Q[:,:i],R[:i,i])
        #2.4: Sketch q_i_p: s_i_p = Θq_i_p
        s_p = my_sketch(sketch_mat, q_p)
        #2.5: Compute the sketched norm r_i,i = ||s_i_p||
        R[i,i] = np.linalg.norm(s_p)
        #2.6: Scale vector s_i = s_i_p/r_i.i
        S[:,i] = s_p/R[i,i]
        #2.7: Scale vector q_i = q_i_p/r_i.i
        Q[:,i] = q_p/R[i,i]

        #print("Algo 2")
        #print("R[i,i]", R[i,i])
    return Q, R

def RGS_testing_2(iter, W, k):
    results = []
    times = []
    for i in range(iter):
        sketch_mat=Theta_Matrix(k,n)
        tic = time.perf_counter()
        GS_Q, GS_R = reduced_QR(W)
        toc = time.perf_counter()
        GS_time = toc - tic
        tic = time.perf_counter()
        RGS_Q, RGS_R = algo_2(W, sketch_mat)
        toc = time.perf_counter()
        RGS_time = toc - tic
        time_diff = GS_time - RGS_time
    #Finding and comparing the accuracy of the two methods
        GS_results = W - mat_mul(GS_Q, GS_R)
        GS_Ortho = np.identity(np.shape(W)[1]) - GS_Q.T @ GS_Q
        RGS_results = W - mat_mul(RGS_Q, RGS_R)
        RGS_Ortho = np.identity(np.shape(W)[1]) - RGS_Q.T @ RGS_Q
        GS_norm = np.linalg.norm(GS_results)/np.linalg.norm(W)
        RGS_norm = np.linalg.norm(RGS_results)/np.linalg.norm(W)
        GS_Onorm = np.linalg.norm(GS_Ortho)/(np.sqrt(np.shape(W)[1]))
        RGS_Onorm = np.linalg.norm(RGS_Ortho)/(np.sqrt(np.shape(W)[1]))
        accuracy = (GS_norm, RGS_norm, GS_Onorm, RGS_Onorm)
    #Setting results
        results.append(accuracy)
        times.append(time_diff)
    return results, times

The following code tests algo_2 against reduced_QR.

In [ ]:
n = 10
m = 5
k = 7
W = np.random.rand(n, m)
iter = 1


#run tests
results, times = RGS_testing_2(iter, W, k)

#print results
for i in range(iter):
    print("On iteration", i)
    print("The accuracy difference was: " + str(results[i]))
    print("The time difference was: " + str(times[i]))
    if times[i] < 0:
        print("Gram Schmidt was quicker than RGS here.")
    elif times[i] == 0:
        print("RGS and GS took the same time here.")
    else:
        print("RGS was quicker than GS here.")
    print("\n")

On iteration 0
The accuracy difference was: (7.947338315729995e-17, 6.0412611954036e-17, 6.599425734868943e-16, 3.0599246930938434e-16)
The time difference was: -0.00013412200496532023
Gram Schmidt was quicker than RGS here.




Unfortunately, it seems that algo_2 runs much slower than rand_QR. However, the accuracy is much better. While the intention of randomized GMRes is to eventually speed up computation time, it is not helpful if that is achieved at the loss of accuracy. For that reason, we will continue to use algo_2.

##Randomized GMRes

###Grigori & Balabanov Algorithms

The RGMRes implementation developed was based on the following two algorithms from the Grigori & Balabanov paper, along with the following minimization step to calculate a new guess after each iteration.

<img src='https://i.imgur.com/ewbMXkZ.png'>

<img src='https://i.imgur.com/bVKQtyD.png'>

<img src='https://i.imgur.com/9gxHgff.png'>

In [ ]:
A = np.random.rand(3,3)
A[:,:3]

array([[0.7670881 , 0.2475551 , 0.57909153],
       [0.41238268, 0.92254861, 0.00900287],
       [0.73929993, 0.53187073, 0.3744976 ]])

Given the above algorithms, and using the Sauer GMRes implementation to guide the logic of this implementation's development, the following RGMRes was developed.

###Randomized GMRes Implementation

The following function is the minimization step of the randomized GMRes algorithm.

In [ ]:
#Parameters: arrays A, Q, R, sketch_mat
#Returns: array x
def find_x(A, Q, R, sketch_mat):
    #Getting sizes of matrices
    n = len(Q)
    m = len(Q[0])

    #Minimize function
    b = np.zeros((m,1))
    b[0] = R[0,0]

    print("b")
    print(b)

    sketched_AQ = my_sketch(sketch_mat, A) #kxn with sketching, nxn without
    sketched_AQ = np.dot(sketched_AQ, Q[:,:m]) #kxm-1 with sketching, nxm-1 without
    sketched_b = my_sketch(sketch_mat, Q) #kxm with sketching, nxm without
    sketched_b = np.dot(sketched_b, b) #kx1 with sketching, mx1 without

    y = np.linalg.lstsq(sketched_AQ, sketched_b)[0] #kxm-1 * y  -  kx1 => y = m-1x1, nxm-1 * y  -  mx1 => m-1x1

    print("sketched_b")
    print(sketched_b)
    print("sketched_AQ")
    print(sketched_AQ)

    print("y")
    print(y)

    x = np.dot(Q[:,:m],y) #nx1

    print("x")
    print(x)

    return x

The following code implements RGMRes. It also defines a my_dot function so the user can decide whether to use sketching or not.

In [ ]:
#RGS-Arnoldi GMRes
#Parameters: nxm array A, nx1 array b, kxn array sketch_mat, bool sketch
#Returns: nxm factor array Q_m, mxm upper triangular factor R_m, result array X
def my_random_gmres(A, b, sketch_mat, sketch = False):
    
    if (sketch):
      def my_sketch(sketch_mat, mat):
        return sketch_mat @ mat
    else:
        def my_sketch(sketch_mat, mat):
          return mat



    #Getting sizes of matrices
    n = len(A)
    m = len(A[0])

    #3.1: Set w_1 = b
    W = b
    W_1 = b
    #3.2: Perform 1st iteration of Algorithm 2
    Q, R = algo_2(W, sketch_mat)
    Q_1, R_1 = reduced_QR(W_1)

    for i in range(1, m):
        #3.3: Compute w_i = Aq_i-1
        w_i = np.dot(A,Q[:,[i-1]])
        W = np.hstack((W, w_i))
        w_i_1 = np.dot(A,Q_1[:,[i-1]])
        W_1 = np.hstack((W_1, w_i_1))

        #3.4: Perform i-th iteration of Algorithm 2
        Q, R = algo_2(W, sketch_mat)
        Q_1, R_1 = reduced_QR(W_1)

        print("Algo 2 Q")
        print(Q)
        print("Reduced QR Q")
        print(Q_1)
        print("Algo 2 R")
        print(R)
        print("Reduced QR R")
        print(R_1)

        #Minimize step to find next x guess
        print("Algo 2 Find x")
        x = find_x(A, Q, R, sketch_mat)
        print("Reduced QR find x")
        x_1 = find_x(A, Q_1, R_1, sketch_mat)

        #save array of errors between each successive guess instead of
        #saving all x guesses

    return Q, R, x, x_1

The following function allows us to easily test the error of x guesses.

In [ ]:
#Function to compare x vectors
def xCompare(x1, x2, xTrue = 0):
  compError = abs(x1 - x2)

The following code tests my_random_gmres.

In [ ]:
size = 3
#A = np.random.rand(size,size)
A = np.array([[-7,8,1],[6,-3,2],[5,-1,-4]])
#x_true = np.random.rand(size,1)
x_true = np.array([[-5],[-6],[9]])
b = np.dot(A,x_true)
sketch_mat = Theta_Matrix(size,size)
sketch = False
Q, R, x, x_1 = my_random_gmres(A, b, sketch_mat, sketch)
print("x true")
print(x_true)
print("RGMRes x")
print(x)
print("reduced QR x")
print(x_1)
#print((x.reshape(size,1)-x_true)/np.linalg.norm(x_true))

Algo 2 Q
[[-0.07211012  0.04481374]
 [ 0.10816519 -0.99274687]
 [-0.99151421 -0.11155884]]
Reduced QR Q
[[-0.07211012  0.04481374]
 [ 0.10816519 -0.99274687]
 [-0.99151421 -0.11155884]]
Algo 2 R
[[55.47071299 -3.79135522]
 [ 0.          2.34711601]]
Reduced QR R
[[55.47071299 -3.79135522]
 [ 0.          2.34711601]]
Algo 2 Find x
b
[[55.47071299]
 [ 0.        ]]
sketched_b
[[ -4.]
 [  6.]
 [-55.]]
sketched_AQ
[[ 0.37857815 -8.36723003]
 [-2.74018472  3.0240054 ]
 [ 3.49734102  1.66305096]]
y
[[-10.92835053]
 [ -1.23858418]]
x
[[ 0.73253912]
 [ 0.0475335 ]
 [10.97378982]]
Reduced QR find x
b
[[55.47071299]
 [ 0.        ]]
sketched_b
[[ -4.]
 [  6.]
 [-55.]]
sketched_AQ
[[ 0.37857815 -8.36723003]
 [-2.74018472  3.0240054 ]
 [ 3.49734102  1.66305096]]
y
[[-10.92835053]
 [ -1.23858418]]
x
[[ 0.73253912]
 [ 0.0475335 ]
 [10.97378982]]
Algo 2 Q
[[-0.07211012  0.04481374 -0.99638941]
 [ 0.10816519 -0.99274687 -0.05247799]
 [-0.99151421 -0.11155884  0.06673981]]
Reduced QR Q
[[-0.07211012  0.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


We are still in the process of debugging the RGMRes code. Ideally, when the code runs properly, we can easily switch between GMRes and RGMRes to compare the two.

#Results & Discussion

We do not have significant results to show since our RGMRes implementation does not yet run properly.

#Conclusion

We do not have significant conclusions to draw since our RGMRes implementation does not yet run properly.

#Bibliography

ali_m. (2015, November 5). *Getting the number of iterations of scipy's gmres iterative method*. Stack Overflow. https://stackoverflow.com/questions/33512081/getting-the-number-of-iterations-of-scipys-gmres-iterative-method

Balabanov, O., & Grigori, L. (2022, January 18). *Randomized Gram-Schmidt process with applications to GMRes.*

Batchelder, W., Matrinez, V., & Vue, J. *NREL - rGS Project.*

Sauer, T. (2018). Generalized Minimum Residual (GMRES) Method. *Numerical Analysis* (3rd ed. pp. 235-239). Pearson.

Wikimedia Foundation. (2022, Feburary 22). *Generalized minimal residual method.* Wikipedia. https://en.wikipedia.org/wiki/Generalized_minimal_residual_method